In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

player = {'Season' : 0, 'Name' : '', 'Position' : '', 'Age' : 0, 'Fee (€ 1Th)' : 0}
player_df = pd.DataFrame(columns=['Season', 'Name', 'Position', 'Age', 'Fee (€ 1Th)'])
stats = {'Season' : 0, 'Name' : '', 'Position' : '', 'Age' : 0, 
         'Competition' : '', 'Appearances' : 0, 'Goals' : 0, 'Assists' : 0, 'Yellow_cards' : 0,
         'Second_yellow_cards' : 0, 'Red_cards' : 0, 'Penalty_goals' : 0, 
         'Minutes_per_goal' : 0, 'Minutes_played' : 0}
stats_df = pd.DataFrame(columns=stats.keys())
stats_df

,Season,Name,Position,Age,Competition,Appearances,Goals,Assists,Yellow_cards,Second_yellow_cards,Red_cards,Penalty_goals,Minutes_per_goal,Minutes_played


In [83]:
for page in range(1, 21):
    url = f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&page={page}'
    header = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    }
    
    seed = np.random.randint(100) 
    np.random.seed(seed) 
    a = np.random.randint(2) 
    time.sleep(a) 

    resp = requests.get(url, headers = header)
    html = BeautifulSoup(resp.text, 'html.parser')

    for item in html.find_all('tr', class_ = ['odd', 'even']):
        player['Name'] = item.find_all('td')[3].text
        player['Position'] =item.find_all('td')[4].text
        age_22 =int(item.find_all('td')[5].text)
    
        item_url = 'https://www.transfermarkt.com' + item.find_all('td')[3].find('a').attrs['href']
        detail_referer = item_url.replace('profil', 'detaillierteleistungsdaten')
        detail_url = detail_referer + '/plus/1'
               
                    
        seed = np.random.randint(100) 
        np.random.seed(seed) 
        a = np.random.randint(2) 
        time.sleep(a)
        
        header = {
            'referer' : 'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'   
        }
        
        item_resp = requests.get(item_url, headers = header)
        
        item_html = BeautifulSoup(item_resp.text, 'html.parser')
        
        for season_item, fee_item in zip(item_html.find_all('div', class_='tm-player-transfer-history-grid__season')[1 : -1], item_html.find_all('div', class_='tm-player-transfer-history-grid__fee')[1 : -1]):            
            fee_col = fee_item.text.strip()
            season_col = season_item.text.strip()
            
            if '/' in season_col:
                season = int('20' + season_col.split('/')[0])
            else:
                season = int(season_col)
                
            player['Season'] = season
            player['Age'] = age_22 - (2022 - season)
            
            try:
                if fee_col[0] == '€' and fee_col.find('m') != -1:
                    fee_ = fee_col.split('.')
                    fee = int(fee_[0][1 : ] + fee_[1][ : -1]) * 10
                    player['Fee (€ 1Th)'] = fee
                elif fee_col[0] == '€' and fee_col.find('Th'):
                    fee_ = fee_col.split('Th')
                    fee = int(fee_[0][1 : ])
                    player['Fee (€ 1Th)'] = fee
                else:
                    continue
#                 print(player)
                player_df = player_df.append(player, ignore_index=True)
            except:
                pass
            
        header_detail = {
            'header' : detail_referer,
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
        }
        
        seed = np.random.randint(100) 
        np.random.seed(seed) 
        a = np.random.randint(2) 
        time.sleep(a)
        
        detail_resp = requests.get(detail_url, headers = header)
        detail_html = BeautifulSoup(detail_resp.text, 'html.parser')
        
        stats['Name'] = player['Name']
        stats['Position'] = player['Position']
        
        for box in detail_html.find_all('div', class_='box')[1 : ]:
            datas = box.find_all('tr', class_=['odd', 'even'])
            
            for data in datas:
                for index, col in enumerate(data.find_all('td')):
                    if index == 0:
                        if '/' in col.text.strip():
                            stats['Season'] = int('20' + col.text.strip().split('/')[0])
                        else:
                            stats['Season'] = int(col.text.strip())
                        stats['Age'] = age_22 - (2022 - stats['Season'])
                    elif index == 2:
                        stats['Competition'] = col.find('a').text.strip()
                    elif index == 4:
                        try:
                            stats['Appearances'] = int(col.find('a').text.strip())
                        except:
                            stats['Appearances'] = 0
                    elif index == 5:
                        try:
                            stats['Goals'] = int(col.text.strip())
                        except:
                            stats['Goals'] = 0
                    elif index == 6:
                        try:
                            stats['Assists'] = int(col.text.strip())
                        except:
                            stats['Assists'] = 0

                    elif index == 10:
                        try:
                            stats['Yellow_cards'] = int(col.text.strip())
                        except:
                            stats['Yellow_cards'] = 0
                    elif index == 11:
                        try:
                            stats['Second_yellow_cards'] = int(col.text.strip())
                        except:
                            stats['Second_yellow_cards'] = 0
                    elif index == 12:
                        try:
                            stats['Red_cards'] = int(col.text.strip())
                        except:
                            stats['Red_cards'] = 0
                    elif index == 13:
                        try:
                            stats['Penalty_goals'] = int(col.text.strip())
                        except:
                            stats['Penalty_goals'] = 0
                    elif index == 14:
                        if "'" in col.text:
                            if (".") in col.text:
                                stats['Minutes_per_goal'] = int(col.text.strip().split('.')[0])
                            else:
                                stats['Minutes_per_goal'] = int(col.text.strip().split("'")[0])
                        else:
                            stats['Minutes_per_goal'] = 0
                    elif index == 15:
                        if "'" in col.text:
                            if "." in col.text:
                                stats['Minutes_played'] = int(col.text.strip().split('.')[0])
                            else:
                                stats['Minutes_played'] = int(col.text.strip().split("'")[0])
                        else:
                            stats['Minutes_played'] = 0
                stats_df = stats_df.append(stats, ignore_index=True)
#                 print(stats)
                
            
    



In [84]:
stats_df

,Season,Name,Position,Age,Competition,Appearances,Goals,Assists,Yellow_cards,Second_yellow_cards,Red_cards,Penalty_goals,Minutes_per_goal,Minutes_played
0,2022,Kylian Mbappé,Centre-Forward,23,Ligue 1,6,7,0,1,0,0,0,71,497
1,2021,Kylian Mbappé,Centre-Forward,22,Ligue 1,35,28,19,10,0,0,4,108,3
2,2020,Kylian Mbappé,Centre-Forward,21,Ligue 1,31,27,7,5,0,0,6,88,2
3,2019,Kylian Mbappé,Centre-Forward,20,Ligue 1,20,18,7,0,0,0,0,84,1
4,2018,Kylian Mbappé,Centre-Forward,19,Ligue 1,29,33,9,6,0,1,1,70,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15340,2019,Giorgian de Arrascaeta,Attacking Midfield,25,Libertadores,11,1,3,0,0,0,0,673,673
15341,2018,Giorgian de Arrascaeta,Attacking Midfield,24,Libertadores,9,3,1,0,0,0,0,252,757
15342,2017,Giorgian de Arrascaeta,Attacking Midfield,23,Copa Sudamericana,2,0,1,0,0,0,0,0,180
15343,2015,Giorgian de Arrascaeta,Attacking Midfield,21,Libertadores,10,1,1,0,0,0,0,745,745


In [85]:
player_df

,Season,Name,Position,Age,Fee (€ 1Th)
0,2018,Kylian Mbappé,Centre-Forward,19,180000
1,2022,Erling Haaland,Centre-Forward,22,60000
2,2019,Erling Haaland,Centre-Forward,19,20000
3,2018,Erling Haaland,Centre-Forward,18,8000
4,2016,Erling Haaland,Centre-Forward,16,100
...,...,...,...,...,...
837,2021,José Sá,Goalkeeper,28,8000
838,2019,José Sá,Goalkeeper,26,2500
839,2015,José Sá,Goalkeeper,22,1200
840,2018,Giorgian de Arrascaeta,Attacking Midfield,24,15000


In [86]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Season       842 non-null    object
 1   Name         842 non-null    object
 2   Position     842 non-null    object
 3   Age          842 non-null    object
 4   Fee (€ 1Th)  842 non-null    object
dtypes: object(5)
memory usage: 33.0+ KB


In [87]:
player_df.describe()

,Season,Name,Position,Age,Fee (€ 1Th)
count,842,842,842,842,842
unique,19,425,13,21,339
top,2019,Ciro Immobile,Centre-Forward,22,15000
freq,133,7,156,117,31


In [88]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15345 entries, 0 to 15344
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Season               15345 non-null  object
 1   Name                 15345 non-null  object
 2   Position             15345 non-null  object
 3   Age                  15345 non-null  object
 4   Competition          15345 non-null  object
 5   Appearances          15345 non-null  object
 6   Goals                15345 non-null  object
 7   Assists              15345 non-null  object
 8   Yellow_cards         15345 non-null  object
 9   Second_yellow_cards  15345 non-null  object
 10  Red_cards            15345 non-null  object
 11  Penalty_goals        15345 non-null  object
 12  Minutes_per_goal     15345 non-null  object
 13  Minutes_played       15345 non-null  object
dtypes: object(14)
memory usage: 1.6+ MB


In [89]:
stats_df.describe()

,Season,Name,Position,Age,Competition,Appearances,Goals,Assists,Yellow_cards,Second_yellow_cards,Red_cards,Penalty_goals,Minutes_per_goal,Minutes_played
count,15345,15345,15345,15345,15345,15345,15345,15345,15345,15345,15345,15345,15345,15345
unique,21,500,13,24,307,47,44,23,16,4,59,24,896,962
top,2021,Cristiano Ronaldo,Centre-Back,20,Champions League,1,0,0,0,0,0,0,0,1
freq,1895,83,2578,1719,1544,3058,8806,9521,8765,14914,14469,13885,8290,1467


In [90]:
stats_df.to_excel('player_stats.xlsx')

In [91]:
player_df.to_excel('player_fee.xlsx')

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [92]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

In [93]:
url = 'https://1xbet.whoscored.com/Statistics'
driver.get(url)

In [131]:
Player_dict = {
    'Season' : 0,
    'Name' : '',
    'Appearances' : '',
    'Minutes_played' : '', 
    'Goals' : '',
    'Assists' : '',
    'Yellow_cards' : '',
    'Red_cards' : '',
    'Shots_per_game' : '',
    'Pass_success_percentage' : '',
    'AerialsWone' : '',
    'MotM' : '',
    'Rating' : ''
}

whoscored_df = pd.DataFrame(columns=Player_dict.keys())
whoscored_df

,Season,Name,Appearances,Minutes_played,Goals,Assists,Yellow_cards,Red_cards,Shots_per_game,Pass_success_percentage,AerialsWone,MotM,Rating


In [134]:

driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[33]/td[1]').text

'Total / Average'

In [99]:
for page in range(142):
    for row in range(1, 11):
        Player_dict['Name'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]/a[1]/span').text.strip()
        driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]/a[1]/span').click()
        t.sleep(2)
        driver.find_element(By.XPATH, f'//*[@id="sub-navigation"]/ul/li[4]/a').click()
        t.sleep(1)
        
        for row in range(1, 50):
            if 'Total' in driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]').text:
                break
            
            if '/' in driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]').text:
                Player_dict['Season'] = int('20' + driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]').text.split('/')[0])
            else:
                try:
                    Player_dict['Season'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]').text)
                except:
                    Player_dict['Season'] = 0
            if '(' in driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[6]').text:
                Player_dict['Appearances'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[6]').text.split('(')[0])
            else:
                try:
                    Player_dict['Appearances'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[6]').text)
                except:
                    Player_dict['Appearances'] = 0
            try:
                Player_dict['Minutes_played'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[7]').text.strip())
            except:
                Player_dict['Minutes_played'] = 0
                
            try:
                Player_dict['Goals'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[8]').text.strip())
            except:
                Player_dict['Goals'] = 0 
            
            try:
                Player_dict['Assists'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[9]').text.strip())
            except:
                Player_dict['Assists'] = 0 
                
            try:
                Player_dict['Yellow_cards'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[10]').text.strip())
            except:
                Player_dict['Yellow_cards'] = 0 
            
            try:
                Player_dict['Red_cards'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[11]').text.strip())
            except:
                Player_dict['Red_cards'] = 0 
            
            try:
                Player_dict['Shots_per_game'] = float(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[12]').text.strip())
            except:
                Player_dict['Shots_per_game'] = 0 
                
            try:
                Player_dict['Pass_success_percentage'] = float(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[13]').text.strip())
            except:
                Player_dict['Pass_success_percentage'] = 0
                
            try:
                Player_dict['AerialsWone'] = float(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[14]').text.strip())
            except:
                Player_dict['AerialsWone'] = 0 
                
            try:
                Player_dict['MotM'] = int(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[15]').text.strip())
            except:
                Player_dict['MotM'] = 0 
                
            try:
                Player_dict['Rating'] = float(driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[16]').text.strip())
            except:
                Player_dict['Rating'] = 0 
            whoscored_df = whoscored_df.append(Player_dict, ignore_index=True)
            
#         print(Player_dict)
        driver.back()
        t.sleep(2)
        driver.back()
        t.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="statistics-paging-summary"]/div/dl[2]/dd[3]').click()
    seed = np.random.randint(100) 
    np.random.seed(seed) 
    a = np.random.randint(3, 5) 
    time.sleep(a)                                                          

In [100]:
for row in range(1, 3):
    Player_dict['Name'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[1]/a[1]/span').text.strip()
    Player_dict['Apps'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[3]').text.strip()
    Player_dict['Mins'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[4]').text.strip()
    Player_dict['Goals'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[5]').text.strip()
    Player_dict['Assists'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[6]').text.strip()
    Player_dict['Yel'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[7]').text.strip()
    Player_dict['Red'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[8]').text.strip()
    Player_dict['SpG'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[9]').text.strip()
    Player_dict['PS%'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[10]').text.strip()
    Player_dict['AAerialsWone'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[11]').text.strip()
    Player_dict['MotM'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[12]').text.strip()
    Player_dict['Rating'] = driver.find_element(By.XPATH, f'//*[@id="player-table-statistics-body"]/tr[{row}]/td[13]').text.strip()

    whoscored_df = whoscored_df.append(Player_dict, ignore_index=True)

    print(Player_dict)

{'Name': 'Fabrizio Angileri', 'Appearances': '5', 'Minutes_played': '324', 'Goals': '-', 'Assists': '-', 'Yellow_cards': '-', 'Red_cards': '2', 'Shots_per_game': '1.4', 'Pass_success_percentage': '85.4', 'AerialsWone': '0.6', 'MotM': '-', 'Rating': '5.71', 'Apps': '4', 'Mins': '322', 'Yel': '1', 'Red': '1', 'SpG': '0.5', 'PS%': '58.6', 'AAerialsWone': '2.8'}
{'Name': 'Léo Jardim', 'Appearances': '5', 'Minutes_played': '324', 'Goals': '-', 'Assists': '-', 'Yellow_cards': '-', 'Red_cards': '2', 'Shots_per_game': '1.4', 'Pass_success_percentage': '85.4', 'AerialsWone': '0.6', 'MotM': '-', 'Rating': '5.66', 'Apps': '6', 'Mins': '540', 'Yel': '-', 'Red': '-', 'SpG': '-', 'PS%': '75.4', 'AAerialsWone': '0.3'}


In [128]:
whoscored_df 

,Name,Appearances,Minutes_played,Goals,Assists,Yellow_cards,Red_cards,Shots_per_game,Pass_success_percentage,AerialsWone,MotM,Rating,AAerialsWone,Apps,Mins,PS%,Red,SpG,Yel
0,Lionel Messi,7,620,3,7,-,-,4.6,83.4,-,4,8.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Neymar,6,533,8,6,2,-,2.6,79.4,-,1,8.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Robert Lewandowski,4,377,6,2,-,-,3.8,83.2,0.8,3,8.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Erling Haaland,6,485,10,1,-,-,4.2,80.5,0.3,2,8.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kylian Mbappé,6,497,7,-,1,-,5.5,86.3,0.7,1,8.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1417,Danny Ward,6,540,-,-,-,-,-,73.9,-,-,5.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1418,Igor Julio,3,291,-,-,1,1,0.3,97.4,0.8,-,5.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1419,Romain Hamouma,5,324,-,-,-,2,1.4,85.4,0.6,-,5.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,Fabrizio Angileri,5,324,-,-,-,2,1.4,85.4,0.6,-,5.71,2.8,4,322,58.6,1,0.5,1


In [127]:
for index, item in enumerate(whoscored_df['Rating']):
    try:
        whoscored_df['Rating'][index] = float(whoscored_df['Rating'][index])
    except:
        print(whoscored_df['Rating'][index])   
        pass

In [122]:
whoscored_df['Shots_per_game'][0]

4.6

In [66]:
whoscored_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1422 entries, 0 to 1421
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          1422 non-null   object
 1   Apps          1422 non-null   object
 2   Mins          1422 non-null   object
 3   Goals         1422 non-null   object
 4   Assists       1422 non-null   object
 5   Yel           1422 non-null   object
 6   Red           1422 non-null   object
 7   SpG           1422 non-null   object
 8   PS%           1422 non-null   object
 9   AAerialsWone  1422 non-null   object
 10  MotM          1422 non-null   object
 11  Rating        1422 non-null   object
dtypes: object(12)
memory usage: 133.4+ KB


In [67]:
whoscored_df.to_excel('whoscored.xlsx')